In [1]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd
import datetime as dt

## Read run time JSON

In [2]:
scene_folder = 'scene-json'
case_folder = '2-cityffd-97' ## todo - JSON folder
header = 'sf_100_dt_hvac_cfd_r2_09_' ## todo - JSON header
CSV_FILE_NAME = 'sf_97_2m_hvac_cfd_r2.csv' ## todo - CSV Schedule file header
hourly_data = {}
for day in range(2, 0, -1):
    for hour in range(1, 25):
#         print(os.path.join(scene_folder, case_folder, header + str(day).zfill(2) + '_' + str(hour).zfill(2) + '_00.json'))
        with open(os.path.join(scene_folder, case_folder, header + str(day).zfill(2) + '_' + str(hour).zfill(2) + '_00.json'), 'r') as f:
            wrf_data = json.load(f)
            for wrf_grid in wrf_data:
                wrf_id = wrf_grid['id']
                dry_bulb = wrf_grid['properties']['dry_bulb_temperature']
                wind_speed = wrf_grid['properties']['wind_speed']
                wind_direction = wrf_grid['properties']['wind_direction']
                if wrf_id not in hourly_data:
                    hourly_data[wrf_id] = [(dry_bulb, wind_speed, wind_direction)]
                else:
                    hourly_data[wrf_id].append((dry_bulb, wind_speed, wind_direction))

In [3]:
hourly_data[1][:12]

[(27.17963791, 0.01390243, 356.84603882),
 (25.17474174, 0.00927262, 0.06849796),
 (23.67122269, 0.00425348, 347.85864258),
 (23.91352463, 0.01051374, 190.95320129),
 (24.07212067, 0.00853058, 217.41415405),
 (23.04794502, 0.14113741, 358.28796387),
 (22.59319687, 0.48090928, 355.41555786),
 (24.54581261, 0.12979895, 349.54541016),
 (28.51670647, 0.00900958, 352.56616211),
 (34.68254089, 0.01663667, 189.30412292),
 (42.72354507, 0.01573932, 352.14364624),
 (50.98550415, 0.016706, 178.63909912)]

## Write 48 hour JSON to 48 hour CSV

In [6]:
CSV_FILE_NAME = 'sf_97_2m_hvac_cfd_r2_day.csv' ## todo
with open(os.path.join(scene_folder, 'run_time_inputs', CSV_FILE_NAME), 'w') as out:
    header_line = []
    for wrf_id in hourly_data:      
        for header in ['drybulb', 'wind_speed', 'wind_dir']:
            header_name = header + ':' + str(wrf_id)
            header_line.append(header_name)
    out.write(','.join(header_line) + '\n')
    for hour in range(24, 48):
        curr_line = []
        for wrf_id in hourly_data:
            curr_line.append(str(round(hourly_data[wrf_id][hour][0], 3)))
            curr_line.append(str(round(hourly_data[wrf_id][hour][1], 3)))
            curr_line.append(str(round(hourly_data[wrf_id][hour][2], 3)))
        out.write(','.join(curr_line) + '\n')
    for hour in range(24):
        curr_line = []
        for wrf_id in hourly_data:
            curr_line.append(str(round(hourly_data[wrf_id][hour][0], 3)))
            curr_line.append(str(round(hourly_data[wrf_id][hour][1], 3)))
            curr_line.append(str(round(hourly_data[wrf_id][hour][2], 3)))
        out.write(','.join(curr_line) + '\n')
        
            

## Write 48 hour JSON to 8760 hour CSV

In [7]:
CSV_FILE_NAME = 'sf_97_2m_hvac_cfd_r2.csv'
with open(os.path.join(scene_folder, 'run_time_inputs', CSV_FILE_NAME), 'w') as out:
    header_line = []
    for wrf_id in hourly_data:      
        for header in ['drybulb', 'wind_speed', 'wind_dir']:
            header_name = header + ':' + str(wrf_id)
            header_line.append(header_name)
    out.write(','.join(header_line) + '\n')
    for day in range(365 // 2):
        for hour in range(48):
            curr_line = []
            for wrf_id in hourly_data:
                curr_line.append(str(round(hourly_data[wrf_id][hour][0], 3)))
                curr_line.append(str(round(hourly_data[wrf_id][hour][1], 3)))
                curr_line.append(str(round(hourly_data[wrf_id][hour][2], 3)))
            out.write(','.join(curr_line) + '\n')
    for hour in range(24):
        curr_line = []
        for wrf_id in hourly_data:
            curr_line.append(str(round(hourly_data[wrf_id][hour][0], 3)))
            curr_line.append(str(round(hourly_data[wrf_id][hour][1], 3)))
            curr_line.append(str(round(hourly_data[wrf_id][hour][2], 3)))
        out.write(','.join(curr_line) + '\n')

In [8]:
SCHEDULE_FILE_NAME = 'sf_97_2m_hvac_cfd_r2.csv'
df_csv_result = pd.read_csv(os.path.join(scene_folder, 'run_time_inputs', SCHEDULE_FILE_NAME), sep=',', encoding='UTF-8')
df_csv_result.head()

,drybulb:1,wind_speed:1,wind_dir:1,drybulb:2,wind_speed:2,wind_dir:2,drybulb:3,wind_speed:3,wind_dir:3,drybulb:4,...,wind_dir:3300,drybulb:3301,wind_speed:3301,wind_dir:3301,drybulb:3302,wind_speed:3302,wind_dir:3302,drybulb:3303,wind_speed:3303,wind_dir:3303
0,27.180,0.014,356.846,27.180,0.014,356.846,27.180,0.014,356.846,27.180,...,266.255,26.884,0.028,270.939,26.061,0.020,224.428,26.762,0.010,285.170
1,25.175,0.009,0.068,25.175,0.009,0.068,25.175,0.009,0.068,25.175,...,249.655,25.834,0.019,231.409,24.770,0.015,83.512,25.270,0.023,230.355
2,23.671,0.004,347.859,23.671,0.004,347.859,23.671,0.004,347.859,23.671,...,183.004,26.597,0.329,181.754,22.955,0.216,90.107,25.200,0.219,125.456
3,23.914,0.011,190.953,23.914,0.011,190.953,23.914,0.011,190.953,23.914,...,185.479,26.935,0.381,177.588,23.636,0.282,83.839,25.333,0.233,117.696
4,24.072,0.009,217.414,24.072,0.009,217.414,24.072,0.009,217.414,24.072,...,249.582,25.006,0.132,185.392,24.691,0.179,74.695,24.421,0.187,148.726


## Separate to smaller files for faster reading

In [9]:
for i in range(34):
    new_result = pd.DataFrame()
    for j in range(1, 101):
        grid_id = i * 100 + j
        for header in ['drybulb', 'wind_speed', 'wind_dir']:
            if not header + ':' + str(grid_id) in df_csv_result:
                continue
            new_result[header + ':' + str(grid_id)] = df_csv_result[header + ':' + str(grid_id)]
    new_result.to_csv(os.path.join(scene_folder, 'run_time_inputs', "sf_97_2m_hvac_cfd_r2_" + str(i) + '.csv')) # todo